In [2]:
import requests
import chardet
import os
import re
import whisper
import torch
from bs4 import BeautifulSoup
from pydub import AudioSegment
from urllib.parse import urlparse

In [ ]:
def get_audio_url_from_podcast(url):
    """
    Extract audio URL and podcast title from an Apple Podcast page.
    """
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch the webpage. Status code: {response.status_code}\nURL: {url}")

    # 自动检测编码
    detected_encoding = chardet.detect(response.content)['encoding']
    print(f"Detected encoding: {detected_encoding}")
    if detected_encoding:
        response.encoding = detected_encoding
    else:
        response.encoding = 'utf-8'  # 默认使用 utf-8 编码

    # 打印前 500 个字符的 HTML 内容进行验证
    #print("Fetched HTML content preview:")
    #print(response.text[:5000])  # 打印部分 HTML 内容

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find audio URL (MP3 or M4A)
    audio_urls = re.findall(r'https://[^\s^"]+(?:\.mp3|\.m4a)', response.text)
    if not audio_urls:
        raise Exception("No audio URLs found on the page.")
    return audio_urls[-1], soup  # Return the last found URL and soup


def get_podcast_title(soup):
    """
    Extract the title of the podcast episode from the HTML soup.
    """
    title_tag = soup.find('title')
    if title_tag:
        title = title_tag.text.strip()
        # 清理多余的后缀
        title = re.sub(r" - .*Apple Podcasts.*", "", title)
        # 处理可能的编码问题
        title = title.encode('latin1').decode('utf-8') if 'charset=iso' in str(soup) else title
        return title
    else:
        return "unknown_podcast"


def format_filename(title):
    """
    Format the title to create a safe filename.
    """
    title = re.sub(r'[^\w\-_\. ]', '_', title)  # 替换特殊字符为 "_"
    return title


def download_audio_file(audio_url, title, output_folder="downloads"):
    """
    Download the audio file from a given URL and save it locally with the given title.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 格式化标题为文件名
    formatted_title = format_filename(title)
    file_extension = os.path.splitext(urlparse(audio_url).path)[1]
    output_path = os.path.join(output_folder, f"{formatted_title}{file_extension}")

    # Stream and save the audio file
    with requests.get(audio_url, stream=True) as response:
        response.raise_for_status()
        with open(output_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

    return output_path


def convert_audio_to_wav(input_file, title):
    """
    Convert audio file (MP3/M4A) to WAV format and save it with the given title.
    """
    formatted_title = format_filename(title)
    output_file = os.path.join(os.path.dirname(input_file), f"{formatted_title}.wav")
    
    if input_file.lower().endswith(".mp3"):
        audio = AudioSegment.from_mp3(input_file)
    elif input_file.lower().endswith(".m4a"):
        audio = AudioSegment.from_file(input_file, "m4a")
    else:
        raise Exception("Unsupported audio format. Only MP3 and M4A are supported.")
    
    audio.export(output_file, format="wav")
    return output_file




# 交互式部分：提示用户输入 Apple Podcast 链接
podcast_url = input("请输入 Apple Podcast 的链接: ")

try:
    # 1. 提取音频 URL 和 HTML 内容
    audio_url, soup = get_audio_url_from_podcast(podcast_url)
    print(f"提取的音频 URL: {audio_url}")

    # 2. 提取标题
    podcast_title = get_podcast_title(soup)
    print(f"提取的博客标题: {podcast_title}")

    # 3. 询问用户是否修改文件名
    custom_title = input(f"请输入文件名（按 n 使用默认标题“{podcast_title}”）: ").strip()
    if custom_title.lower() == 'n' or not custom_title:
        custom_title = podcast_title  # 使用默认值
    print(f"最终使用的文件名: {custom_title}")

    # 4. 下载音频文件
    downloaded_file = download_audio_file(audio_url, custom_title)
    print(f"音频文件已下载: {downloaded_file}")

    # 5. 转换为 WAV 格式
    wav_file = convert_audio_to_wav(downloaded_file, custom_title)
    print(f"音频文件已转换为 WAV 格式: {wav_file}")


except Exception as e:
    print(f"处理过程中发生错误: {e}")

In [4]:
import whisper
import gc

def load_whisper_model_v3(model_name="large-v3"):
    """
    Load the Whisper model (large-v3) from Hugging Face.
    """
    print(f"Loading Whisper model: {model_name}...")
    model = whisper.load_model(model_name)
    print(f"Model {model_name} loaded successfully.")
    return model


def transcribe_audio_v3(model, audio_path, task="transcribe", language=None,verbose=True):
    """
    Transcribe audio using Whisper model (large-v3).
    """
    print(f"Transcribing audio: {audio_path}...")
    result = model.transcribe(audio_path, task=task, language=language, verbose=verbose)#增加了最后一个verbose
    print("Transcription completed.")
    return result


def save_transcription_as_srt_v3(transcription, audio_path):
    """
    Save transcription result as SRT file.
    """
    srt_file_path = os.path.splitext(audio_path)[0] + ".srt"
    with open(srt_file_path, "w", encoding="utf-8") as srt_file:
        for segment in transcription['segments']:
            start = segment['start']
            end = segment['end']
            text = segment['text']
            # Write to SRT format
            srt_file.write(f"{segment['id'] + 1}\n")
            srt_file.write(f"{format_timestamp(start)} --> {format_timestamp(end)}\n")
            srt_file.write(f"{text}\n\n")
    print(f"Transcription saved as SRT: {srt_file_path}")
    return srt_file_path


def format_timestamp(seconds):
    """
    Format timestamp for SRT file.
    """
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{int(seconds):02},{milliseconds:03}"



# 主程序：模块二集成
if __name__ == "__main__":
    # 假设模块一已经完成，直接获取 WAV 文件路径
    wav_file_path = wav_file  # 上一模块生成的结果

    # 确保文件存在
    if not wav_file_path or not os.path.exists(wav_file_path):
        print(f"错误：音频文件 {wav_file_path} 不存在！请确保模块一已正确运行。")
    else:
        try:
            # 1. 加载 Whisper large-v3 模型
            model = load_whisper_model_v3("large-v3")

            # 2. 转录音频
            transcription_result = transcribe_audio_v3(model, wav_file_path)

            # 3. 保存转录结果为 SRT 文件
            srt_file_path = save_transcription_as_srt_v3(transcription_result, wav_file_path)

            print(f"转录完成！字幕文件保存在: {srt_file_path}")

        except Exception as e:
            print(f"处理过程中发生错误: {e}")

        finally:
            # 显存释放
            print("Releasing GPU memory...")
            del model  # 删除模型对象
            torch.cuda.empty_cache()  # 释放未使用的显存
            gc.collect()  # 强制进行垃圾回收
            print("GPU memory released.")


Loading Whisper model: large-v3...
Model large-v3 loaded successfully.
Transcribing audio: downloads/2月份月度对话.wav...
Releasing GPU memory...


KeyboardInterrupt: 